In [1]:
import torch
import os
from pathlib import Path
import json

/Users/michalisdikaiopoulos/Desktop/Master/Autumn2025/Advanced Topics in Deep Learning/Assignments/Assignment_2/Winning-Ticket-Hypothesis-Study/.venv/lib/python3.9/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
CHECKPOINTS = [
    "./new_models/lottery_ticket_sst2_sparsity60_seed42.pt",
    "./new_models/lottery_ticket_sst2_sparsity60_seed123.pt",
]

In [3]:
def verify_checkpoint(checkpoint_path):
    """Thoroughly inspect a checkpoint file."""

    print("\n" + "="*70)
    print(f"📦 Inspecting: {os.path.basename(checkpoint_path)}")
    print("="*70)

    if not os.path.exists(checkpoint_path):
        print(f"❌ File not found: {checkpoint_path}")
        return None

    # Load checkpoint
    try:
        ckpt = torch.load(checkpoint_path, map_location="cpu")
    except Exception as e:
        print(f"❌ Failed to load checkpoint: {e}")
        return None

    # --- Checkpoint Structure ---
    print("\n🔍 Checkpoint Keys:")
    for key in ckpt.keys():
        print(f"   - {key}")

    # --- Model State Dict ---
    if "model_state_dict" in ckpt:
        state_dict = ckpt["model_state_dict"]
        print(f"\n🧠 Model State Dict: {len(state_dict)} keys")

        # Categorize keys
        bert_keys = [k for k in state_dict.keys() if k.startswith("bert.")]
        classifier_keys = [k for k in state_dict.keys() if k.startswith("classifier")]
        other_keys = [k for k in state_dict.keys() if not k.startswith("bert.") and not k.startswith("classifier")]

        print(f"   - BERT encoder keys: {len(bert_keys)}")
        print(f"   - Classifier keys: {len(classifier_keys)}")
        if other_keys:
            print(f"   - Other keys: {len(other_keys)}")

        # Show classifier shape
        if classifier_keys:
            print(f"\n   📊 Classifier layers:")
            for key in classifier_keys:
                shape = state_dict[key].shape
                print(f"      {key}: {shape}")

        # Sample BERT weights
        print(f"\n   🔬 Sample BERT weights (first 3):")
        for key in list(bert_keys)[:3]:
            shape = state_dict[key].shape
            print(f"      {key}: {shape}")

    # --- Mask Information ---
    if "mask_dict" in ckpt:
        mask_dict = ckpt["mask_dict"]
        print(f"\n🎭 Mask Dict: {len(mask_dict)} masks found")

        # Count masked parameters
        total_params = 0
        total_nonzero = 0

        for key, mask in mask_dict.items():
            total_params += mask.numel()
            total_nonzero += mask.sum().item()

        sparsity = (1 - total_nonzero / total_params) * 100
        print(f"   - Total parameters: {total_params:,}")
        print(f"   - Non-zero parameters: {total_nonzero:,}")
        print(f"   - Sparsity: {sparsity:.2f}%")

        # Show some mask keys
        print(f"\n   🔬 Sample mask keys (first 3):")
        for key in list(mask_dict.keys())[:3]:
            shape = mask_dict[key].shape
            nonzero = mask_dict[key].sum().item()
            total = mask_dict[key].numel()
            print(f"      {key}: {shape} -> {nonzero}/{total} active ({nonzero/total*100:.1f}%)")
    else:
        print(f"\n⚠️  No mask_dict found - this might be a dense model (sparsity 0)")

    # --- Training Metadata ---
    metadata_keys = ["task", "seed", "sparsity", "final_accuracy", "best_val_acc"]
    print(f"\n📋 Training Metadata:")
    for key in metadata_keys:
        if key in ckpt:
            print(f"   - {key}: {ckpt[key]}")

    # Check for any other metadata
    other_metadata = [k for k in ckpt.keys() if k not in ["model_state_dict", "mask_dict"] + metadata_keys]
    if other_metadata:
        print(f"\n   Additional metadata:")
        for key in other_metadata:
            value = ckpt[key]
            if isinstance(value, (int, float, str, bool)):
                print(f"   - {key}: {value}")
            else:
                print(f"   - {key}: {type(value)}")

    # --- Verification Summary ---
    print(f"\n✅ Verification Summary:")
    has_encoder = "model_state_dict" in ckpt and len([k for k in ckpt["model_state_dict"].keys() if k.startswith("bert.")]) > 0
    has_classifier = "model_state_dict" in ckpt and len([k for k in ckpt["model_state_dict"].keys() if k.startswith("classifier")]) > 0
    has_mask = "mask_dict" in ckpt

    print(f"   ✓ BERT encoder weights: {'✅' if has_encoder else '❌'}")
    print(f"   ✓ Classifier weights: {'✅' if has_classifier else '❌'}")
    print(f"   ✓ Lottery ticket mask: {'✅' if has_mask else '⚠️  (dense model)'}")

    return {
        "path": checkpoint_path,
        "has_encoder": has_encoder,
        "has_classifier": has_classifier,
        "has_mask": has_mask,
        "sparsity": sparsity if has_mask else 0.0,
    }

In [4]:
def verify_all_checkpoints(checkpoint_list):
    """Verify all checkpoints and create summary report."""

    print("\n" + "🔬"*35)
    print("LOTTERY TICKET CHECKPOINT VERIFICATION")
    print("🔬"*35)

    results = []
    for ckpt_path in checkpoint_list:
        result = verify_checkpoint(ckpt_path)
        if result:
            results.append(result)

    # --- Summary Table ---
    print("\n" + "="*70)
    print("📊 SUMMARY TABLE")
    print("="*70)
    print(f"{'Checkpoint':<50} {'Encoder':<10} {'Mask':<10} {'Sparsity':<10}")
    print("-"*70)

    for result in results:
        name = os.path.basename(result["path"])
        encoder = "✅" if result["has_encoder"] else "❌"
        mask = "✅" if result["has_mask"] else "⚠️"
        sparsity = f"{result['sparsity']:.1f}%"
        print(f"{name:<50} {encoder:<10} {mask:<10} {sparsity:<10}")

    print("\n✅ Verification complete!\n")
    return results

In [5]:
def check_transfer_compatibility(source_checkpoint, target_task):
    """Check if a checkpoint is compatible for transfer to target task."""

    print(f"\n🔄 Checking transfer compatibility:")
    print(f"   Source: {os.path.basename(source_checkpoint)}")
    print(f"   Target: {target_task.upper()}")

    ckpt = torch.load(source_checkpoint, map_location="cpu")
    state_dict = ckpt.get("model_state_dict", {})

    # Check BERT encoder
    bert_keys = [k for k in state_dict.keys() if k.startswith("bert.")]
    if not bert_keys:
        print("   ❌ No BERT encoder found!")
        return False

    print(f"   ✅ BERT encoder: {len(bert_keys)} parameters")

    # Check mask (for lottery tickets)
    if "mask_dict" in ckpt:
        mask_dict = ckpt["mask_dict"]
        total = sum(m.numel() for m in mask_dict.values())
        nonzero = sum(m.sum().item() for m in mask_dict.values())
        print(f"   ✅ Lottery mask: {(1-nonzero/total)*100:.1f}% sparse")
    else:
        print(f"   ⚠️  No mask found (dense model)")

    # Check classifier (will be reinitialized anyway)
    classifier_keys = [k for k in state_dict.keys() if k.startswith("classifier")]
    if classifier_keys:
        print(f"   ℹ️  Classifier found (will be reinitialized for {target_task})")

    print(f"   ✅ Compatible for transfer learning!")
    return True

In [7]:
if __name__ == "__main__":
    # Verify all checkpoints
    results = verify_all_checkpoints(CHECKPOINTS)


🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬
LOTTERY TICKET CHECKPOINT VERIFICATION
🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬

📦 Inspecting: lottery_ticket_qqp_sparsity70_seed42.pt

🔍 Checkpoint Keys:
   - model_state_dict
   - sparsity
   - seed
   - history

🧠 Model State Dict: 203 keys
   - BERT encoder keys: 0
   - Classifier keys: 0
   - Other keys: 203

   🔬 Sample BERT weights (first 3):

⚠️  No mask_dict found - this might be a dense model (sparsity 0)

📋 Training Metadata:
   - seed: 42
   - sparsity: 0.7

   Additional metadata:
   - history: <class 'dict'>

✅ Verification Summary:
   ✓ BERT encoder weights: ❌
   ✓ Classifier weights: ❌
   ✓ Lottery ticket mask: ⚠️  (dense model)

📦 Inspecting: lottery_ticket_sst2_sparsity60_seed42.pt

🔍 Checkpoint Keys:
   - model_state_dict
   - sparsity
   - seed
   - history

🧠 Model State Dict: 203 keys
   - BERT encoder keys: 0
   - Classifier keys: 0
   - Other keys: 203

   🔬 Sample BERT weights (first 3):

⚠️  No mask_dict found - this might be a 

In [8]:
import os
import json

# Directory containing your JSON files
folder_path = "./fewshot_models"  # change this to your folder

combined_data = []

# Loop through all .json files
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        filepath = os.path.join(folder_path, filename)
        with open(filepath, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                combined_data.append(data)
            except json.JSONDecodeError as e:
                print(f"⚠️ Skipping {filename}: invalid JSON ({e})")

# Convert combined data to a single JSON string
combined_json_str = json.dumps(combined_data, ensure_ascii=False, indent=2)

# Print or save the combined string
print(combined_json_str)

# Optional: save to file
# with open("combined.json", "w", encoding="utf-8") as f:
#     f.write(combined_json_str)


[
  {
    "source_checkpoint": "./models/lottery_ticket_qqp_sparsity70_seed42.pt",
    "target_task": "sst2",
    "mode": "headonly",
    "seed": 42,
    "train_acc": 0.609375,
    "val_acc": 0.625,
    "examples": 128
  },
  {
    "source_checkpoint": "./models/ensemble_ticket_sst2_qqp_sparsity47_seed42.pt",
    "target_task": "sst2",
    "mode": "full",
    "seed": 42,
    "train_acc": 0.8359375,
    "val_acc": 0.8125,
    "examples": 128
  },
  {
    "source_checkpoint": "./models/lottery_ticket_qqp_sparsity70_seed999.pt",
    "target_task": "sst2",
    "mode": "headonly",
    "seed": 999,
    "train_acc": 0.4921875,
    "val_acc": 0.4375,
    "examples": 128
  },
  {
    "source_checkpoint": "./models/lottery_ticket_qqp_sparsity70_seed123.pt",
    "target_task": "sst2",
    "mode": "headonly",
    "seed": 123,
    "train_acc": 0.546875,
    "val_acc": 0.5625,
    "examples": 128
  },
  {
    "source_checkpoint": "./models/lottery_ticket_qqp_sparsity70_seed42.pt",
    "target_task":

In [10]:
import os
import json
import pandas as pd

results = []

# Folder where your summary JSONs are saved
summary_dir = "./models"

for fname in os.listdir(summary_dir):
    if not fname.endswith("_summary.json"):
        continue

    path = os.path.join(summary_dir, fname)
    with open(path) as f:
        data = json.load(f)

    task = data["task"]
    seed = data["seed"]
    sparsity = data["sparsity"]
    hist = data["history"]

    # Get first and last entries
    train_acc_dense = hist["train_acc"][0]
    val_acc_dense = hist["val_acc"][0]
    train_acc_target = hist["train_acc"][-1]
    val_acc_target = hist["val_acc"][-1]

    results.append({
        "task": task,
        "seed": seed,
        "target_sparsity": sparsity,
        "train_acc_dense": train_acc_dense,
        "val_acc_dense": val_acc_dense,
        "train_acc_target": train_acc_target,
        "val_acc_target": val_acc_target,
    })

# Create DataFrame
df = pd.DataFrame(results)

# Sort for readability
df = df.sort_values(by=["task", "seed", "target_sparsity"]).reset_index(drop=True)

# Print summary
print(df)

# Optionally, save to CSV
out_path = os.path.join(summary_dir, "lottery_ticket_results_table.csv")
df.to_csv(out_path, index=False)
print(f"\n✅ Results table saved to {out_path}")


    task  seed  target_sparsity  train_acc_dense  val_acc_dense  \
0    qqp    42              0.0         0.822447       0.861995   
1    qqp    42              0.7         0.822447       0.861995   
2    qqp   123              0.0         0.820962       0.861283   
3    qqp   123              0.7         0.820962       0.861283   
4    qqp   999              0.0         0.820012       0.860392   
5    qqp   999              0.7         0.820012       0.860392   
6   sst2    42              0.0         0.911921       0.923165   
7   sst2    42              0.6         0.911921       0.923165   
8   sst2   123              0.0         0.908477       0.918578   
9   sst2   123              0.6         0.908477       0.918578   
10  sst2   999              0.0         0.913005       0.916284   
11  sst2   999              0.6         0.913005       0.916284   

    train_acc_target  val_acc_target  
0           0.822447        0.861995  
1           0.979513        0.865736  
2          

In [11]:
#!/usr/bin/env python3
import os
import json
import re
import pandas as pd

fewshot_dir = "./fewshot_models"
records = []

for fname in os.listdir(fewshot_dir):
    if not fname.endswith(".json") or not fname.startswith("fewshot_"):
        continue

    path = os.path.join(fewshot_dir, fname)
    with open(path) as f:
        data = json.load(f)

    # --- Extract main info from JSON ---
    target_task = data.get("target_task", "unknown")
    mode = data.get("mode", "unknown")
    seed = data.get("seed", None)
    train_acc = data.get("train_acc", None)
    val_acc = data.get("val_acc", None)
    examples = data.get("examples", None)
    source_checkpoint = data.get("source_checkpoint", "")

    # --- Parse source checkpoint name ---
    source_type = "unknown"
    source_tasks = "unknown"
    sparsity = None

    m = re.search(r"(lottery_ticket|ensemble_ticket)_([a-zA-Z0-9_]+)_sparsity(\d+)_seed(\d+)", source_checkpoint)
    if m:
        source_type = m.group(1).replace("_ticket", "")
        source_tasks = m.group(2).replace("_", "+")  # e.g., sst2_qqp → sst2+qqp
        sparsity = float(m.group(3)) / 100.0

    records.append({
        "target_task": target_task,
        "source_type": source_type,
        "source_tasks": source_tasks,
        "sparsity": sparsity,
        "seed": seed,
        "mode": mode,
        "train_acc": train_acc,
        "val_acc": val_acc,
        "examples": examples,
    })

df = pd.DataFrame(records)
df = df.sort_values(by=["target_task", "source_type", "sparsity", "mode", "seed"]).reset_index(drop=True)

print(df)

out_path = os.path.join(fewshot_dir, "fewshot_results_table.csv")
df.to_csv(out_path, index=False)
print(f"\n✅ Few-shot results table saved to {out_path}")


   target_task source_type source_tasks  sparsity  seed      mode  train_acc  \
0          qqp    ensemble     sst2+qqp      0.47    42      full   0.640625   
1          qqp    ensemble     sst2+qqp      0.47   123      full   0.640625   
2          qqp    ensemble     sst2+qqp      0.47   999      full   0.640625   
3          qqp    ensemble     sst2+qqp      0.47    42  headonly   0.640625   
4          qqp    ensemble     sst2+qqp      0.47   123  headonly   0.640625   
5          qqp    ensemble     sst2+qqp      0.47   999  headonly   0.578125   
6          qqp     lottery         sst2      0.00    42      full   0.640625   
7          qqp     lottery         sst2      0.00   123      full   0.640625   
8          qqp     lottery         sst2      0.00   999      full   0.640625   
9          qqp     lottery         sst2      0.00    42  headonly   0.640625   
10         qqp     lottery         sst2      0.00   123  headonly   0.640625   
11         qqp     lottery         sst2 

In [12]:
#!/usr/bin/env python3
import os
import re
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

# Directory containing all model .pt files
model_dir = "./models"

# --- Helper functions ---
def flatten_weights(state_dict):
    """Return dict of flattened tensors for all linear layers."""
    flat = {}
    for k, v in state_dict.items():
        if not isinstance(v, torch.Tensor):
            continue
        # Ignore biases and embeddings
        if v.ndim >= 2 and "embeddings" not in k and "LayerNorm" not in k:
            flat[k] = v.flatten()
    return flat

def cosine_similarity_models(path_a, path_b):
    """Compute mean cosine similarity between two model state dicts."""
    wa = torch.load(path_a, map_location="cpu")["model_state_dict"]
    wb = torch.load(path_b, map_location="cpu")["model_state_dict"]

    fa, fb = flatten_weights(wa), flatten_weights(wb)
    sims = []
    for k in fa:
        if k in fb and fa[k].numel() == fb[k].numel():
            sims.append(F.cosine_similarity(fa[k], fb[k], dim=0).item())

    return float(np.mean(sims)) if sims else np.nan

# --- Collect ticket paths ---
tickets = [f for f in os.listdir(model_dir) if f.startswith("lottery_ticket") and f.endswith(".pt")]
tickets = [f for f in tickets if not "sparsity0" in f]  # exclude dense models

sst2_tickets = [f for f in tickets if "_sst2_" in f]
qqp_tickets  = [f for f in tickets if "_qqp_" in f]

records = []

# --- Compare same-seed SST2 vs QQP ---
for sst_file in sst2_tickets:
    m = re.search(r"sparsity(\d+)_seed(\d+)", sst_file)
    if not m:
        continue
    sparsity_sst2, seed = int(m.group(1)) / 100.0, int(m.group(2))

    # Find matching qqp file with same seed
    qqp_file = next((f for f in qqp_tickets if f"seed{seed}" in f), None)
    if qqp_file is None:
        continue

    m2 = re.search(r"sparsity(\d+)_seed(\d+)", qqp_file)
    sparsity_qqp = int(m2.group(1)) / 100.0

    path_sst2 = os.path.join(model_dir, sst_file)
    path_qqp = os.path.join(model_dir, qqp_file)

    sim = cosine_similarity_models(path_sst2, path_qqp)

    records.append({
        "seed": seed,
        "sst2_sparsity": sparsity_sst2,
        "qqp_sparsity": sparsity_qqp,
        "cosine_similarity": sim,
        "sst2_model": sst_file,
        "qqp_model": qqp_file,
    })

# --- Create DataFrame ---
df = pd.DataFrame(records)
df = df.sort_values(by="seed").reset_index(drop=True)

print(df)

out_path = os.path.join(model_dir, "cross_task_cosine_similarity.csv")
df.to_csv(out_path, index=False)
print(f"\n✅ Saved similarity table to {out_path}")


   seed  sst2_sparsity  qqp_sparsity  cosine_similarity  \
0    42            0.6           0.7           0.933109   
1   123            0.6           0.7           0.932258   
2   999            0.6           0.7           0.932612   

                                  sst2_model  \
0   lottery_ticket_sst2_sparsity60_seed42.pt   
1  lottery_ticket_sst2_sparsity60_seed123.pt   
2  lottery_ticket_sst2_sparsity60_seed999.pt   

                                  qqp_model  
0   lottery_ticket_qqp_sparsity70_seed42.pt  
1  lottery_ticket_qqp_sparsity70_seed123.pt  
2  lottery_ticket_qqp_sparsity70_seed999.pt  

✅ Saved similarity table to ./models/cross_task_cosine_similarity.csv
